In [ ]:
from langchain_ollama import ChatOllama

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

In [ ]:

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm

chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

In [ ]:
# Graph state
class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

In [ ]:
# Nodes
def generate_joke(state: State) -> dict[str, str]:
    """First LLM call to generate initial joke"""

    msg = llm.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}


def check_punchline(state: State) -> str:
    """Gate function to check if the joke has a punchline"""

    # Simple check - does the joke contain "?" or "!"
    if "?" in state["joke"] or "!" in state["joke"]:
        return "Pass"
    return "Fail"


def improve_joke(state: State) -> dict[str, str]:
    """Second LLM call to improve the joke"""

    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")
    return {"improved_joke": msg.content}


def polish_joke(state: State) -> dict[str, str]:
    """Third LLM call for final polish"""

    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")
    return {"final_joke": msg.content}


In [ ]:
# Build workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("generate_joke", generate_joke)
workflow.add_node("improve_joke", improve_joke)
workflow.add_node("polish_joke", polish_joke)

# Add edges to connect nodes
workflow.add_edge(START, "generate_joke")
workflow.add_conditional_edges(
    "generate_joke", check_punchline, {"Fail": "improve_joke", "Pass": END}
)
workflow.add_edge("improve_joke", "polish_joke")
workflow.add_edge("polish_joke", END)

In [ ]:
# Compile
chain = workflow.compile()

# Show workflow
display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke
state = chain.invoke({"topic": "cats"})
print("Initial joke:")
print(state["joke"])
print("\n--- --- ---\n")
if "improved_joke" in state:
    print("Improved joke:")
    print(state["improved_joke"])
    print("\n--- --- ---\n")

    print("Final joke:")
    print(state["final_joke"])
else:
    print("Joke failed quality gate - no punchline detected!")

# My Solution

## AI Agents

- Orchestrator
- Software Architect
    - Code formatters - code styling, spacing, identation
    - Refactor Tools: Suggest code improvements & restructuring
        - Functions are too long
        - Duplicate code detection 
- Security Engineer
    - Security scanner
        - Hardcoded secrets,
        - SQL injection patterns 
    - CVE database
        - CVE API
        - Detect top 10 attacks
        - GitHub Advisory Database API
    - Depracated tools and uses
        - Check changes in requirements, package.json, check if they are outdated 
- Performance Engineer
    - Profiling tool: nested loops, inefficient queries
    - Benchmark tools: cyclomatic complexity, nesting depth
- Tech Writer
    - Check for missing docstrings/comments 
    - Generate Docs from code
- Tech lead
    - Merge tools
    - Notification tool

**Optional**

- Code Reviewer
    - AST parser
    - Linting Tools

## Define Tools

## Define Agents

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

orchastrator = create_react_agent(llm, tools=[])
security_agent = create_react_agent(llm, tools=[])
software_architect_agent = create_react_agent(llm, tools=[])
performance_agent = create_react_agent(llm, tools=[])
tech_writer_agent = create_react_agent(llm, tools=[])
tech_lead = create_react_agent(llm, tools=[])
software_engineer_lead = create_react_agent(llm, tools=[])

In [ ]:
# def check_packages(): pass

# sub_graph = StateGraph(State)
# sub_graph.add_node("Security Engineer", security_agent)
# sub_graph.add_node("Check Packages", check_packages)
# sub_graph.add_edge(START, "Security Engineer")
# sub_graph.add_edge("Security Engineer", "Check Packages")
# sub_graph.add_edge("Check Packages", END)
# sub_graph = sub_graph.compile()



In [ ]:
workflow = StateGraph(State)

# Add nodes
workflow.add_node("Project Manager", orchastrator)
workflow.add_node("Security Engineer", security_agent)
workflow.add_node("Software Architect", software_architect_agent)
workflow.add_node("Performance Engineer", performance_agent)
workflow.add_node("Technical Writer", tech_writer_agent)
workflow.add_node("Tech Lead", tech_lead)
workflow.add_node("Software Engineer", software_engineer_lead)


# Add edges to connect nodes
workflow.add_edge(START, "Project Manager")
workflow.add_edge("Project Manager", "Security Engineer")
workflow.add_edge("Project Manager", "Software Architect")
workflow.add_edge("Project Manager", "Performance Engineer")
workflow.add_edge("Project Manager", "Technical Writer")



workflow.add_edge("Security Engineer", "Tech Lead")
workflow.add_edge("Software Architect", "Tech Lead")
workflow.add_edge("Performance Engineer", "Tech Lead")
workflow.add_edge("Technical Writer", "Tech Lead")

workflow.add_edge("Tech Lead", END)

workflow.add_conditional_edges(
    "Tech Lead", approve_changes, {"Rejected Merge": "Software Engineer", "Approved Merge": END}
)

workflow.add_edge("Software Engineer", END)


In [ ]:
def approve_changes():
    pass

In [ ]:
# Compile
chain = workflow.compile()

# Show workflow
display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:


llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

search_agent = create_react_agent(llm, tools=[tavily_tool])

def search_node(state: State) -> Command[Literal["supervisor"]]:
    result = search_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="search")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


web_scraper_agent = create_react_agent(llm, tools=[scrape_webpages])


def web_scraper_node(state: State) -> Command[Literal["supervisor"]]:
    result = web_scraper_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="web_scraper")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )


research_supervisor_node = make_supervisor_node(llm, ["search", "web_scraper"])